In [10]:
import os
import conda
import pandas as pd
import numpy as np
import json
import pint
# from shapely.geometry import Polygon as Poly

pd.options.display.max_columns = 250

conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib

import matplotlib.pyplot as plt
import matplotlib.cm
import matplotlib.colors as colors

import geopandas
from geopandas.tools import sjoin
import geoplot as gplt
import geoplot.crs as gcrs

import folium

In [14]:
from pint import UnitRegistry
ureg = UnitRegistry()

# Databases preparation

## Land Cover

Take Land Cover table from EnviroAtlas, regarding land cover, and reconciliate at HUC8 level since chl-a data for TSI estimation is only available at HUC8 spatial scale. Additionaly, the work of Lee and Bakshi, 2019 (Energy-Water-CO2 Nexus of Fossil Fuel Based Power Generation, book carbon management) support the use of HUC8 scale for Techno-Ecological synergy (TES) analysis.

The fields of the databse are described below:

| Field Name |                                           Data Layer Name                                           |
|:----------:|:---------------------------------------------------------------------------------------------------:|
|   N_INDEX  |                                      Percent natural land cover                                     |
|    PAGT    | Percentage of WBD 12-digit Hydrologic Unit Land area that is classified as agriculture (21, 81, 82) |
|   PFOR90   |                                  Percent forest and woody wetlands                                  |
|   PWETL95  |                                 Percent emergent herbaceous wetlands                                |
|    PFOR    |                                            Percent forest                                           |
|    PWETL   |                                           Percent wetlands                                          |
|    PDEV    |                                        Percent developed area                                       |
|    PAGC    |                                           Percent cropland                                          |
|    PAGP    |                                           Percent pasture                                           |


HUC12 is transformed in HUC8 for data spatial consistency deleting the last 4 digits of the code (Federal Standards and Procedures for the National Watershaed Boundary Dataset (WBD), USGS, 2013). 

To restimate the land cover percentajes the values of each item for the same HUC8 watershed are summed, and the percentajes recalculated again over the new total value for HUC8

In [154]:
LandCover_df = pd.read_csv('LandCover/National_metric_tables_in_CSV/CONUS_metrics_May2019_CSV/LandCover.csv', converters={'HUC_12': lambda x: str(x)}) 
LandCover_df

,HUC_12,N_INDEX,PFOR,PWETL,PDEV,PAGT,PAGP,PAGC,PFOR90,PWETL95
0,010100020101,100.000000,81.643204,11.392500,0.000000,0.000000,0.000000,0.000000,92.245598,0.789988
1,010100020102,100.000000,74.108200,12.300500,0.000000,0.000000,0.000000,0.000000,86.143501,0.265201
2,010100020103,100.000000,78.816101,13.675100,0.000000,0.000000,0.000000,0.000000,92.095398,0.395778
3,010100020104,100.000000,72.776901,7.579820,0.000000,0.000000,0.000000,0.000000,80.097702,0.259059
4,010100020105,100.000000,74.281403,13.116500,0.000000,0.000000,0.000000,0.000000,81.182098,6.215810
5,010100020201,99.839203,80.429001,5.992480,0.160756,0.000000,0.000000,0.000000,86.216202,0.205273
6,010100020202,100.000000,82.489899,12.551700,0.000000,0.000000,0.000000,0.000000,94.915604,0.125930
7,010100020203,100.000000,79.267097,11.043100,0.000000,0.000000,0.000000,0.000000,89.944801,0.365425
8,010100020204,99.994400,80.136299,9.918930,0.000000,0.005647,0.000000,0.005647,87.546501,2.508770
9,010100020301,99.774200,83.703796,9.742270,0.225844,0.000000,0.000000,0.000000,93.268997,0.177132


In [155]:
# File loaded again to avoid overwritting of the LandCover_df variable
LandCover_df = pd.read_csv('LandCover/National_metric_tables_in_CSV/CONUS_metrics_May2019_CSV/LandCover.csv', converters={'HUC_12': lambda x: str(x)}) 

# Convert HUC12 to HUC8
LandCover_df['HUC_8'] = LandCover_df['HUC_12'].map(lambda x: str(x)[:-4])
LandCover_df = LandCover_df.drop('HUC_12',1) #0 for rows, 1 for columns

# LandCover_df['HUC_8'] = LandCover_df['HUC_8'].astype(int)
LandCover_df['Total'] = 100

LandCover_df

,N_INDEX,PFOR,PWETL,PDEV,PAGT,PAGP,PAGC,PFOR90,PWETL95,HUC_8,Total
0,100.000000,81.643204,11.392500,0.000000,0.000000,0.000000,0.000000,92.245598,0.789988,01010002,100
1,100.000000,74.108200,12.300500,0.000000,0.000000,0.000000,0.000000,86.143501,0.265201,01010002,100
2,100.000000,78.816101,13.675100,0.000000,0.000000,0.000000,0.000000,92.095398,0.395778,01010002,100
3,100.000000,72.776901,7.579820,0.000000,0.000000,0.000000,0.000000,80.097702,0.259059,01010002,100
4,100.000000,74.281403,13.116500,0.000000,0.000000,0.000000,0.000000,81.182098,6.215810,01010002,100
5,99.839203,80.429001,5.992480,0.160756,0.000000,0.000000,0.000000,86.216202,0.205273,01010002,100
6,100.000000,82.489899,12.551700,0.000000,0.000000,0.000000,0.000000,94.915604,0.125930,01010002,100
7,100.000000,79.267097,11.043100,0.000000,0.000000,0.000000,0.000000,89.944801,0.365425,01010002,100
8,99.994400,80.136299,9.918930,0.000000,0.005647,0.000000,0.005647,87.546501,2.508770,01010002,100
9,99.774200,83.703796,9.742270,0.225844,0.000000,0.000000,0.000000,93.268997,0.177132,01010002,100


In [156]:
# Grouped and values sumed, but not percetages yet!
LandCover_dfGrouped = LandCover_df.groupby(['HUC_8']).sum()
LandCover_dfGrouped

,N_INDEX,PFOR,PWETL,PDEV,PAGT,PAGP,PAGC,PFOR90,PWETL95,Total
HUC_8,,,,,,,,,,
01010002,4097.643600,3321.519814,282.046980,2.071450,0.284989,0.088163,0.196827,3583.079910,20.487259,4100
01010003,2291.215496,1733.613396,297.340113,29.957569,178.826970,39.092481,139.734379,2012.386116,18.567774,2500
01010004,5987.979889,4391.402512,917.224541,94.488543,517.531725,77.320543,440.211255,5279.747490,28.879693,6600
01010005,1033.428600,711.407206,244.454398,46.463798,220.107512,30.664473,189.442966,950.056610,5.804847,1300
01010006,2194.817802,1476.533783,317.074897,5.078495,0.103808,0.069891,0.033917,1775.495705,18.113087,2200
01010007,3599.394402,2724.958996,331.773067,0.500214,0.105245,0.039239,0.066006,3034.023071,22.709279,3600
01010008,1202.609093,1020.640102,80.317040,34.136468,63.254378,11.198781,52.055658,1095.480297,5.476792,1300
01010009,445.784084,323.178200,76.634000,32.029460,122.186329,28.046620,94.139681,394.393005,5.419244,600
01010010,562.933506,389.770494,138.588140,36.024910,301.041601,38.929170,262.112300,523.732201,4.626198,900


In [157]:
# Percetages calculation
LandCover_dfHUC8 = LandCover_dfGrouped
LandCover_dfHUC8.update(LandCover_dfHUC8.iloc[:, :-1].div(LandCover_dfHUC8.Total, 0))
LandCover_dfHUC8.update(LandCover_dfHUC8.iloc[:, :-1].mul(100))
LandCover_dfHUC8['Total'] = LandCover_dfHUC8['PFOR'] + LandCover_dfHUC8['PWETL'] + LandCover_dfHUC8['PDEV'] + LandCover_dfHUC8['PAGC'] + LandCover_dfHUC8['PAGP']
LandCover_dfHUC8

,N_INDEX,PFOR,PWETL,PDEV,PAGT,PAGP,PAGC,PFOR90,PWETL95,Total
HUC_8,,,,,,,,,,
01010002,99.942527,81.012678,6.879195,0.050523,0.006951,0.002150,0.004801,87.392193,0.499689,87.949347
01010003,91.648620,69.344536,11.893605,1.198303,7.153079,1.563699,5.589375,80.495445,0.742711,89.589518
01010004,90.726968,66.536402,13.897342,1.431645,7.841390,1.171523,6.669868,79.996174,0.437571,89.706779
01010005,79.494508,54.723631,18.804184,3.574138,16.931347,2.358806,14.572536,73.081278,0.446527,94.033295
01010006,99.764446,67.115172,14.412495,0.230841,0.004719,0.003177,0.001542,80.704350,0.823322,81.763227
01010007,99.983178,75.693305,9.215919,0.013895,0.002923,0.001090,0.001833,84.278419,0.630813,84.926042
01010008,92.508392,78.510777,6.178234,2.625882,4.865721,0.861445,4.004281,84.267715,0.421292,92.180619
01010009,74.297347,53.863033,12.772333,5.338243,20.364388,4.674437,15.689947,65.732168,0.903207,92.337993
01010010,62.548167,43.307833,15.398682,4.002768,33.449067,4.325463,29.123589,58.192467,0.514022,96.158335


In [158]:
# LandCover_dfHUC8['HUC_8']=LandCover_dfHUC8.index
LandCover_dfHUC8.reset_index(level=0, inplace=True)
LandCover_dfHUC8['HUC_6'] = LandCover_dfHUC8['HUC_8'].map(lambda x: str(x)[:-2])
LandCover_dfHUC8

,HUC_8,N_INDEX,PFOR,PWETL,PDEV,PAGT,PAGP,PAGC,PFOR90,PWETL95,Total,HUC_6
0,01010002,99.942527,81.012678,6.879195,0.050523,0.006951,0.002150,0.004801,87.392193,0.499689,87.949347,010100
1,01010003,91.648620,69.344536,11.893605,1.198303,7.153079,1.563699,5.589375,80.495445,0.742711,89.589518,010100
2,01010004,90.726968,66.536402,13.897342,1.431645,7.841390,1.171523,6.669868,79.996174,0.437571,89.706779,010100
3,01010005,79.494508,54.723631,18.804184,3.574138,16.931347,2.358806,14.572536,73.081278,0.446527,94.033295,010100
4,01010006,99.764446,67.115172,14.412495,0.230841,0.004719,0.003177,0.001542,80.704350,0.823322,81.763227,010100
5,01010007,99.983178,75.693305,9.215919,0.013895,0.002923,0.001090,0.001833,84.278419,0.630813,84.926042,010100
6,01010008,92.508392,78.510777,6.178234,2.625882,4.865721,0.861445,4.004281,84.267715,0.421292,92.180619,010100
7,01010009,74.297347,53.863033,12.772333,5.338243,20.364388,4.674437,15.689947,65.732168,0.903207,92.337993,010100
8,01010010,62.548167,43.307833,15.398682,4.002768,33.449067,4.325463,29.123589,58.192467,0.514022,96.158335,010100
9,01010011,82.613998,47.460098,27.962299,2.086710,15.299300,6.346910,8.952400,74.841904,0.580498,92.808418,010100


In [159]:
# Save the new database
LandCover_dfHUC8.to_csv('DatabasesClean/LandCover_HUC8.csv', index=False)

## 2017 Agricultural Census

The 2017 Agricultural Census is used to determine the distribution of crops on croplands. However, the data provided by the Agricultural Census have a spatial resolution of HUC6 corresponding to each hydrological unit HUC8 to determine the distribution of crops in each portion of crop land corresponding to each hydrological unit HUC8. Area is in acres (2017 census of Agriculture. Watersheds. U.S. Department of Agriculture, 2019).

If two or more crops were harvested from the same land during the year (double cropping), the acres were counted for each crop. Therefore, the total acres of all crops harvested could exceed the acres of cropland harvested. No double cropping is allowed for hay or fruit and nut crops.


In [160]:
# Load csv file
AgriCensus_df = pd.read_csv('AgriCensus2017Watershed/wate_all_tablesCLEANED_TRASPOSED.csv')

AgriCensus_df

,HUC_6,Year,Corn,Soybeans,Small grains,Cotton,Rice,Vegetables,Orchards,Greenhouse,Other
0,H010100,2017,1833.0,1254.0,37038.0,NaN,NaN,51854.0,137.0,11.0,22151.0
1,H010200,2017,9257.0,NaN,1170.0,NaN,NaN,3453.0,412.0,58.0,33209.0
2,H010300,2017,14719.0,NaN,293.0,NaN,NaN,758.0,356.0,195.0,59311.0
3,H010400,2017,4002.0,NaN,NaN,NaN,NaN,1069.0,1059.0,66.0,24189.0
4,H010500,2017,163.0,NaN,NaN,NaN,NaN,1158.0,435.0,154.0,55529.0
5,H010600,2017,3639.0,NaN,170.0,NaN,NaN,3813.0,857.0,2181.0,38624.0
6,H010700,2017,4894.0,NaN,NaN,NaN,NaN,4409.0,2983.0,1417.0,41120.0
7,H010801,2017,19289.0,NaN,NaN,NaN,NaN,1685.0,980.0,208.0,99533.0
8,H010802,2017,13153.0,457.0,523.0,NaN,NaN,15501.0,2829.0,4282.0,62847.0
9,H010900,2017,4433.0,NaN,6.0,NaN,NaN,7313.0,1328.0,3782.0,38678.0


In [161]:
# AgriCensus_df.dropna(axis='index', how ='any', subset=['HUC_6'])
# AgriCensus_df.dropna(axis='index', how ='any', subset=['HUC_6']).to_csv('AgriCensus2017Watershed/wate_all_tablesCLEANED_TRASPOSED.csv',index=False)

In [162]:
# Percetages calculation
AgriCensus_df['Total'] = AgriCensus_df[['Corn', 'Soybeans', 'Small grains', 'Cotton', 'Rice', 'Vegetables', 'Orchards', 'Greenhouse', 'Other']].sum(axis=1)
# AgriCensus_df.iloc[:, 1:].astype(float)

AgriCensus_df

,HUC_6,Year,Corn,Soybeans,Small grains,Cotton,Rice,Vegetables,Orchards,Greenhouse,Other,Total
0,H010100,2017,1833.0,1254.0,37038.0,NaN,NaN,51854.0,137.0,11.0,22151.0,114278.0
1,H010200,2017,9257.0,NaN,1170.0,NaN,NaN,3453.0,412.0,58.0,33209.0,47559.0
2,H010300,2017,14719.0,NaN,293.0,NaN,NaN,758.0,356.0,195.0,59311.0,75632.0
3,H010400,2017,4002.0,NaN,NaN,NaN,NaN,1069.0,1059.0,66.0,24189.0,30385.0
4,H010500,2017,163.0,NaN,NaN,NaN,NaN,1158.0,435.0,154.0,55529.0,57439.0
5,H010600,2017,3639.0,NaN,170.0,NaN,NaN,3813.0,857.0,2181.0,38624.0,49284.0
6,H010700,2017,4894.0,NaN,NaN,NaN,NaN,4409.0,2983.0,1417.0,41120.0,54823.0
7,H010801,2017,19289.0,NaN,NaN,NaN,NaN,1685.0,980.0,208.0,99533.0,121695.0
8,H010802,2017,13153.0,457.0,523.0,NaN,NaN,15501.0,2829.0,4282.0,62847.0,99592.0
9,H010900,2017,4433.0,NaN,6.0,NaN,NaN,7313.0,1328.0,3782.0,38678.0,55540.0


In [163]:
AgriCensus_df.update(AgriCensus_df.iloc[:, 2:-1].div(AgriCensus_df.Total, 0))
AgriCensus_df.update(AgriCensus_df.iloc[:, 2:-1].mul(100))
AgriCensus_df['Sum'] = AgriCensus_df.iloc[:, 2:-1].sum(axis=1)
AgriCensus_df

,HUC_6,Year,Corn,Soybeans,Small grains,Cotton,Rice,Vegetables,Orchards,Greenhouse,Other,Total,Sum
0,H010100,2017,1.603983,1.097324,32.410438,NaN,NaN,45.375313,0.119883,0.009626,19.383433,114278.0,100.0
1,H010200,2017,19.464244,NaN,2.460102,NaN,NaN,7.260455,0.866292,0.121954,69.826952,47559.0,100.0
2,H010300,2017,19.461339,NaN,0.387402,NaN,NaN,1.002221,0.470700,0.257827,78.420510,75632.0,100.0
3,H010400,2017,13.170973,NaN,NaN,NaN,NaN,3.518183,3.485272,0.217212,79.608359,30385.0,100.0
4,H010500,2017,0.283779,NaN,NaN,NaN,NaN,2.016052,0.757325,0.268111,96.674733,57439.0,100.0
5,H010600,2017,7.383735,NaN,0.344940,NaN,NaN,7.736791,1.738901,4.425371,78.370262,49284.0,100.0
6,H010700,2017,8.926910,NaN,NaN,NaN,NaN,8.042245,5.441147,2.584682,75.005016,54823.0,100.0
7,H010801,2017,15.850281,NaN,NaN,NaN,NaN,1.384609,0.805292,0.170919,81.788898,121695.0,100.0
8,H010802,2017,13.206884,0.458872,0.525143,NaN,NaN,15.564503,2.840590,4.299542,63.104466,99592.0,100.0
9,H010900,2017,7.981635,NaN,0.010803,NaN,NaN,13.167087,2.391069,6.809507,69.639899,55540.0,100.0


In [164]:
# Delete H from HUC_6 columns
AgriCensus_df['HUC_6'] = AgriCensus_df['HUC_6'].map(lambda x: str(x)[1:])
AgriCensus_df

,HUC_6,Year,Corn,Soybeans,Small grains,Cotton,Rice,Vegetables,Orchards,Greenhouse,Other,Total,Sum
0,010100,2017,1.603983,1.097324,32.410438,NaN,NaN,45.375313,0.119883,0.009626,19.383433,114278.0,100.0
1,010200,2017,19.464244,NaN,2.460102,NaN,NaN,7.260455,0.866292,0.121954,69.826952,47559.0,100.0
2,010300,2017,19.461339,NaN,0.387402,NaN,NaN,1.002221,0.470700,0.257827,78.420510,75632.0,100.0
3,010400,2017,13.170973,NaN,NaN,NaN,NaN,3.518183,3.485272,0.217212,79.608359,30385.0,100.0
4,010500,2017,0.283779,NaN,NaN,NaN,NaN,2.016052,0.757325,0.268111,96.674733,57439.0,100.0
5,010600,2017,7.383735,NaN,0.344940,NaN,NaN,7.736791,1.738901,4.425371,78.370262,49284.0,100.0
6,010700,2017,8.926910,NaN,NaN,NaN,NaN,8.042245,5.441147,2.584682,75.005016,54823.0,100.0
7,010801,2017,15.850281,NaN,NaN,NaN,NaN,1.384609,0.805292,0.170919,81.788898,121695.0,100.0
8,010802,2017,13.206884,0.458872,0.525143,NaN,NaN,15.564503,2.840590,4.299542,63.104466,99592.0,100.0
9,010900,2017,7.981635,NaN,0.010803,NaN,NaN,13.167087,2.391069,6.809507,69.639899,55540.0,100.0


In [165]:
# Save the new database
AgriCensus_df.to_csv('DatabasesClean/Agricensus.csv', index=False)

## Vegetation nutrient uptake

To determine the nutrients uptake of each type of crop data from USDA (2009) is considered (Waste Management Field Handbook). 

For croplands, the land is divided in the portion occupied by corn, soybeans, small grains, cotton, rice,	vegetables, orchards, greenhouse and other crops (namely oil crops, sugar crops, and fruits). 

For pasture lands the average nutrient uptake and crop yield including the following plants is considered:

    -Alfalfa
    -Switchgrass
    -Wheatgrass

For forests lands the nutrient uptake and crop yield of Northern hardwoods is considered

For developed aread null nutrient uptake is considered

For corn, soybeans, cotton, rice and orchards their specific nutrient uptake values are used. For small grains, vegetables, greenhouse crops, pasture crops and forest average values including the most representatve species are used. (USDA, 2009. Waste Management Field Handbook)b

In [8]:
PlantNutrientUptake_df =  pd.read_csv('PlantNutrientUptake/PlantNutrientUptake.csv', index_col='Crop')

For pasture lands the average nutrient uptake and crop yield including the following plants is considered:

    -Alfalfa
    -Switchgrass
    -Wheatgrass

In [167]:
Pasture_df = PlantNutrientUptake_df.loc[['Alfalfa','Switchgrass','Wheatgrass']]
Pasture_df = Pasture_df.mean(axis=0)
Pasture_df.to_csv('DatabasesClean/PasturePlantNutrientUptake.csv', index=True, index_label=['Item'], header=['Value'])
Pasture_df

ConversionFactor    2000.000000
YieldAcre              2.666667
PoundsAcre          5333.333333
N                      1.606667
P                      0.196667
K                      2.150000
Ca                     0.680000
Mg                     0.250000
S                      0.175000
Cu                     0.000800
Mn                     0.005500
Zn                     0.005300
dtype: float64

For forests lands the nutrient uptake and crop yield of Northern hardwoods is considered

In [168]:
Forests_df = PlantNutrientUptake_df.loc[['Northern hardwoods']]
Forests_df = Forests_df.mean(axis=0)
Forests_df.to_csv('DatabasesClean/ForestsPlantNutrientUptake.csv', index=True, index_label=['Item'], header=['Value'])
Forests_df

ConversionFactor      2000.00
YieldAcre               50.00
PoundsAcre          100000.00
N                        0.20
P                        0.02
K                        0.10
Ca                       0.29
Mg                        NaN
S                         NaN
Cu                        NaN
Mn                        NaN
Zn                        NaN
dtype: float64

For small grains the average nutrient uptake and crop yield including the following plants is considered:

    -Wheat
    -WheatStraw
    -Oats
    -OatsStraw
    -Barley
    -BarleyStraw
    -Rye
    -RyeStraw

In [169]:
SmallGrains_df = PlantNutrientUptake_df.loc[['Wheat','WheatStraw','Oats','OatsStraw','Barley','BarleyStraw','Rye','RyeStraw']]
SmallGrains_df = SmallGrains_df.mean(axis=0)
SmallGrains_df.to_csv('DatabasesClean/SmallGrainsPlantNutrientUptake.csv', index=True, index_label=['Item'], header=['Value'])
SmallGrains_df

ConversionFactor    1024.500000
YieldAcre             25.750000
PoundsAcre          2630.000000
N                      1.310000
P                      0.252500
K                      0.812500
Ca                     0.170000
Mg                     0.140000
S                      0.201250
Cu                     0.004613
Mn                     0.006525
Zn                     0.003300
dtype: float64

For vegetables and greehouse the average nutrient uptake and crop yield including the following plants is considered:

    -Bell peppers
    -Beans
    -Cabbage
    -Carrots
    -Cassava
    -Celery
    -Cucumbers
    -Lettuce
    -Onions
    -Peas
    -Potatoes
    -Snap beans
    -Sweet corn
    -Sweet potatoes
    -Table beets
    -Cantaloupe

In [170]:
Vegetables_Greenhouse_df = PlantNutrientUptake_df.loc[['Bell peppers','Beans','Cabbage','Carrots','Cassava','Celery','Cucumbers','Lettuce',
                                                       'Onions','Peas','Potatoes','Snap beans','Sweet corn','Sweet potatoes','Table beets',
                                                       'Cantaloupe']]
Vegetables_Greenhouse_df = Vegetables_Greenhouse_df.mean(axis=0)
Vegetables_Greenhouse_df.to_csv('DatabasesClean/Vegetables_GreenhousePlantNutrientUptake.csv', index=True, index_label=['Item'], header=['Value'])
Vegetables_Greenhouse_df

ConversionFactor     1875.062500
YieldAcre            1104.062500
PoundsAcre          21718.750000
N                       0.744375
P                       0.138125
K                       0.505000
Ca                      0.071000
Mg                      0.084286
S                       0.096000
Cu                      0.000275
Mn                      0.001175
Zn                      0.001040
dtype: float64

For other crops, the average nutrient uptake and crop yield considered includes the most representative crop types not considered in other items, namely oil crops, sugar crops, and fruits:

    -Flax
    -FlaxStraw
    -Oil palm
    -Oil palmStraw
    -Peanuts
    -PeanutsStraw
    -Rapeseed
    -RapeseedStraw
    -Soybeans
    -SoybeansStraw
    -Sunflower
    -SunflowerStraw
    -Apples
    -Bananas
    -Coconuts
    -Grapes
    -Oranges
    -Peaches
    -Pineapple
    -Tomatoes
    -Sugarcane

In [171]:
OtherCrops_df = PlantNutrientUptake_df.loc[['Flax','FlaxStraw','Oil palm','Oil palmStraw','Peanuts','PeanutsStraw','Rapeseed','RapeseedStraw',
                                            'Soybeans','SoybeansStraw','Sunflower','SunflowerStraw','Apples','Bananas','Coconuts','Grapes',
                                            'Oranges','Peaches','Pineapple','Tomatoes','Sugarcane']]
OtherCrops_df = OtherCrops_df.mean(axis=0)
OtherCrops_df.to_csv('DatabasesClean/OtherCropsPlantNutrientUptake.csv', index=True, index_label=['Item'], header=['Value'])
OtherCrops_df

ConversionFactor     1248.571429
YieldAcre            4284.838571
PoundsAcre          18290.952381
N                       1.990476
P                       0.349524
K                       1.180000
Ca                      0.415000
Mg                      0.219048
S                       0.194667
Cu                      0.001067
Mn                      0.006575
Zn                      0.002614
dtype: float64

## Wetlands nutrients regulation

The phosporus uptake by wetlands is considered as 0.77 gP·m-2·year-1 based in the data reported by Kadlec, 2016 (Kadlec, R.H., 2016 Large Constructed Wetlands for Phosphorus Control: A Review. Water, 8, 243)

In [172]:
Wetlands_Preg = 0.77*1E-3/0.0002471052 #Kg/Acre

In [16]:
Wetlands_Preg = 0.77*1E-3/0.0002471052*(ureg.kg/ureg.acre)
Wetlands_Preg

3.1160817336098154 <Unit('kilogram / acre')>

# Watershed retrieval

In [17]:
latitude = 40
longitude = -98

In [18]:
input_point = np.array([float(latitude), float(longitude)]) 

#input_point = np.array([39.014908, -98.010465])

input_point_df = pd.DataFrame(
    {'Name': ['CAFO1'],
     'Latitude': [input_point[0]],
     'Longitude': [input_point[1]]})

#    poly  = geopandas.GeoDataFrame.from_file('cereslibrary/GIS/watershed/huc8sum.shp')
poly  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
point = geopandas.GeoDataFrame(input_point_df, geometry=geopandas.points_from_xy(input_point_df.Longitude, input_point_df.Latitude))

poly.crs = {'init' :'epsg:4326'}
point.crs = {'init' :'epsg:4326'}

pointInPolys = sjoin(point, poly, how='left')

# HUC8ContPoint = (pointInPolys['HUC_8']).astype(str)

# HUC8_NARS_NLA_FINAL_P  = geopandas.GeoDataFrame.from_file('cereslibrary/GIS/HUC8_NARS_NLA_FINAL_P/HUC8_NARS_NLA_FINAL_P.shp')
# HUC8_NARS_NLA_FINAL_N  = geopandas.GeoDataFrame.from_file('cereslibrary/GIS/HUC8_NARS_NLA_FINAL_N/HUC8_NARS_NLA_FINAL_N.shp')

# HUC8_NARS_NLA_FINAL_P.crs = {'init' :'epsg:4326'}
# HUC8_NARS_NLA_FINAL_N.crs = {'init' :'epsg:4326'}


# =============================================================================
# GIS data retrieval
# =============================================================================

# HUC8_NARS_NLA_FINAL_P_index = HUC8_NARS_NLA_FINAL_P.set_index('HUC_8', drop=False)
# HUC8_NARS_NLA_FINAL_N_index = HUC8_NARS_NLA_FINAL_N.set_index('HUC_8', drop=False)

# TP_GIS = HUC8_NARS_NLA_FINAL_P_index.loc[HUC8ContPoint, 'PTL']
# NH4_GIS = HUC8_NARS_NLA_FINAL_N_index.loc[HUC8ContPoint, 'NH4']

HUC8ContPoint = pointInPolys['HUC_8'].values[0]
HUC8ContPoint

'10250016'

In [19]:
HUC6ContPoint = HUC8ContPoint[:6]
HUC6ContPoint

'102500'

In [20]:
poly['HUC_8'] = poly['HUC_8'].astype(str)
AreaHUC8 = poly.set_index('HUC_8', drop=False)[['AREA_ACRES','HUC_8']]
AreaHUC8.to_csv('DatabasesClean/AreaHUC8.csv')
AreaHUC8


,AREA_ACRES,HUC_8
HUC_8,,
04150306,5.512482e+05,04150306
05010001,1.648037e+06,05010001
05050002,1.078369e+06,05050002
05080001,1.598491e+06,05080001
05090203,9.003527e+05,05090203
05120111,1.291943e+06,05120111
05120115,6.713196e+05,05120115
05140102,9.461146e+05,05140102
05140206,6.001628e+05,05140206


In [177]:
poly

,HUC_8,HU_8_NAME,AREA_ACRES,AREA_SQKM,STATES,STORET_COU,STORET_RES,NWIS_COUNT,WSA_CPL_TO,WSA_CPL__1,WSA_CPL__2,WSA_CPL__3,WSA_NAP_TO,WSA_NAP__1,WSA_NAP__2,WSA_NAP__3,WSA_NPL_TO,WSA_NPL__1,WSA_NPL__2,WSA_NPL__3,WSA_SAP_TO,WSA_SAP__1,WSA_SAP__2,WSA_SAP__3,WSA_SPL_TO,WSA_SPL__1,WSA_SPL__2,WSA_SPL__3,WSA_TPL_TO,WSA_TPL__1,WSA_TPL__2,WSA_TPL__3,WSA_UMW_TO,WSA_UMW__1,WSA_UMW__2,WSA_UMW__3,WSA_WMT_TO,WSA_WMT__1,WSA_WMT__2,WSA_WMT__3,WSA_XER_TO,WSA_XER__1,WSA_XER__2,WSA_XER__3,NLA_CGP_TO,NLA_CGP__1,NLA_CGP__2,NLA_CGP__3,NLA_CPE_TO,NLA_CPE__1,NLA_CPE__2,NLA_CPE__3,NLA_GPM_TO,NLA_GPM__1,NLA_GPM__2,NLA_GPM__3,NLA_GPN_TO,NLA_GPN__1,NLA_GPN__2,NLA_GPN__3,NLA_NGL_TO,NLA_NGL__1,NLA_NGL__2,NLA_NGL__3,NLA_SAM_TO,NLA_SAM__1,NLA_SAM__2,NLA_SAM__3,NLA_SEP_TO,NLA_SEP__1,NLA_SEP__2,NLA_SEP__3,NLA_SGL_TO,NLA_SGL__1,NLA_SGL__2,NLA_SGL__3,NLA_TMP_TO,NLA_TMP__1,NLA_TMP__2,NLA_TMP__3,NLA_WMT_TO,NLA_WMT__1,NLA_WMT__2,NLA_WMT__3,NLA_XER_TO,NLA_XER__1,NLA_XER__2,NLA_XER__3,NLCD11A,NLCD11P,NLCD12A,NLCD12P,NLCD21A,NLCD21P,NLCD22A,NLCD22P,NLCD23A,NLCD23P,NLCD24A,NLCD24P,NLCD31A,NLCD31P,NLCD41A,NLCD41P,NLCD42A,NLCD42P,NLCD43A,NLCD43P,NLCD52A,NLCD52P,NLCD71A,NLCD71P,NLCD81A,NLCD81P,NLCD82A,NLCD82P,NLCD90A,NLCD90P,NLCD95A,NLCD95P,LIKELY_TO_,HAS_N_LIMI,HAS_P_LIMI,HAS_N_MONI,HAS_P_MONI,HAS_N_DMR_,HAS_P_DMR_,ATTAINS_30,ATTAINS_TM,Shape_Leng,Shape_Area,geometry
0,04150306,St. Regis,5.512482e+05,2230.822092,NY,5.0,156.0,4.0,0.0,0.0,0.0,0.0,329.0,441.0,8.2,15.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,674.0,1174.0,16.5,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,828.0,1410.0,24.0,102.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67.4973,0.030497,0.0000,0.000000,25.6329,0.011582,2.6937,0.001217,0.3033,0.000137,0.0558,0.000025,1.3869,0.000627,1121.7753,0.506844,362.9259,0.163978,50.8455,0.022973,64.2096,0.029011,20.5236,0.009273,83.8224,0.037873,33.8562,0.015297,352.4211,0.159232,25.3251,0.011442,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,10.0,2.739500,0.253010,POLYGON ((-74.43569948699991 44.34982683300007...
1,05010001,Upper Allegheny,1.648037e+06,6669.368806,"NY,PA",20.0,1681.0,25.0,0.0,0.0,0.0,0.0,329.0,441.0,8.2,15.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,674.0,1174.0,16.5,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,828.0,1410.0,24.0,102.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.3739,0.008902,0.0000,0.000000,170.2944,0.025534,38.0439,0.005704,14.2290,0.002133,2.5335,0.000380,16.0839,0.002412,4270.6368,0.640331,302.5584,0.045365,485.9163,0.072857,331.2000,0.049659,51.3369,0.007697,566.8920,0.084999,217.1736,0.032563,129.0528,0.019350,14.1255,0.002118,17.0,12.0,2.0,7.0,13.0,15.0,15.0,12.0,6.0,6.173681,0.724405,POLYGON ((-79.06824495299992 41.77853391200006...
2,05050002,Middle New,1.078369e+06,4364.005019,"VA,WV",70.0,355.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,296.0,535.0,17.8,24.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.1091,0.005753,0.0000,0.000000,193.1715,0.044261,79.0614,0.018115,34.8129,0.007977,5.5368,0.001269,5.3442,0.001225,2902.8690,0.665131,213.4917,0.048917,82.2726,0.018851,3.9078,0.000895,116.5257,0.026699,685.1295,0.156983,14.9175,0.003418,1.6668,0.000382,0.5238,0.000120,32.0,11.0,0.0,21.0,22.0,23.0,22.0,0.0,4.0,4.452832,0.443767,POLYGON ((-81.52040862099994 37.19589234200004...
3,05080001,Upper Great Miami,1.598491e+06,6468.862509,"IN,OH",1.0,3.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1750.0,3210.0,165.0,338.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

# HUC8 Databases differences

In [178]:
# HUC8 Databases differences
Pruebas_AreaHUC8 = pd.DataFrame(AreaHUC8['HUC_8'])
Pruebas_AreaHUC8 = Pruebas_AreaHUC8.reset_index(drop=True)
Pruebas_LandCover_dfHUC8 = pd.DataFrame(pd.read_csv('DatabasesClean/LandCover_HUC8.csv', converters={'HUC_8': lambda x: str(x), 'HUC_6': lambda x: str(x)})['HUC_8'])
merged = Pruebas_AreaHUC8.merge(Pruebas_LandCover_dfHUC8, indicator=True, how='outer')
merged['HUC_6'] = merged['HUC_8'].map(lambda x: str(x)[:-2])
merged[merged['_merge'] == 'left_only'].to_csv('DatabasesClean/diference.csv')
merged[merged['_merge'] == 'left_only']

,HUC_8,_merge,HUC_6
155,02010001,left_only,020100
316,18030008,left_only,180300
341,18020117,left_only,180201
427,18020107,left_only,180201
482,18060011,left_only,180600
533,18020119,left_only,180201
550,09030004,left_only,090300
646,09010001,left_only,090100
674,09020313,left_only,090203
682,10160010,left_only,101600


# Estimation of the nutrients uptaked by the watershed

Due to problemas with data reconcilitation between EnviroAtlas Land Cover and Agricensus databases, and HUC8 shapefile from EPA NPDAT database, there are no available data for the following HUC8 watersheds:

02010001, 18030008, 18020117, 18020107, 18060011, 18020119, 09030004, 09010001, 09020313, 010160010, 18060012, 02010003, 18100200, 18020102, 18020106, 18020103, 18020101, 18010111, 02010006, 02010007, 02010008, 02060008, 02060009, 02060007, 02060010, 03030001, 18020105, 18020114, 18020112, 18020109, 02010002, 01110000, 10010002, 01010001, 18020113, 18020108, 18020110, 02010005, 18020127, 18020124, 18040005, 18020120, 18030011, 14010006, 18060001, 03020106, 18020118, 18040004, 10010001, 02010004

APLICABLE TO AGRICENSUS AND LAND COVER DATASETS

In [21]:
LandCover_dfHUC8 = pd.read_csv('DatabasesClean/LandCover_HUC8_reconciliated.csv', converters={'HUC_8': lambda x: str(x), 'HUC_6': lambda x: str(x)})
LandCover_dfHUC8 = LandCover_dfHUC8.set_index('HUC_8')
LandCover_dfHUC8

,N_INDEX,PFOR,PWETL,PDEV,PAGT,PAGP,PAGC,PFOR90,PWETL95,Total,HUC_6
HUC_8,,,,,,,,,,,
01010002,99.942527,81.012678,6.879195,0.050523,0.006951,0.002150,0.004801,87.392193,0.499689,87.949347,010100
01010003,91.648620,69.344536,11.893605,1.198303,7.153079,1.563699,5.589375,80.495445,0.742711,89.589518,010100
01010004,90.726968,66.536402,13.897342,1.431645,7.841390,1.171523,6.669868,79.996174,0.437571,89.706779,010100
01010005,79.494508,54.723631,18.804184,3.574138,16.931347,2.358806,14.572536,73.081278,0.446527,94.033295,010100
01010006,99.764446,67.115172,14.412495,0.230841,0.004719,0.003177,0.001542,80.704350,0.823322,81.763227,010100
01010007,99.983178,75.693305,9.215919,0.013895,0.002923,0.001090,0.001833,84.278419,0.630813,84.926042,010100
01010008,92.508392,78.510777,6.178234,2.625882,4.865721,0.861445,4.004281,84.267715,0.421292,92.180619,010100
01010009,74.297347,53.863033,12.772333,5.338243,20.364388,4.674437,15.689947,65.732168,0.903207,92.337993,010100
01010010,62.548167,43.307833,15.398682,4.002768,33.449067,4.325463,29.123589,58.192467,0.514022,96.158335,010100


In [22]:
LandCover_Point = LandCover_dfHUC8.loc[HUC8ContPoint]
LandCover_Point

N_INDEX        42.16
PFOR         1.89975
PWETL        1.25649
PDEV         4.08933
PAGT         53.7506
PAGP         2.18905
PAGC         51.5616
PFOR90       3.08338
PWETL95    0.0728693
Total        60.9962
HUC_6         102500
Name: 10250016, dtype: object

In [23]:
AgriCensus_dfHUC6 = pd.read_csv('DatabasesClean/Agricensus_reconciliated.csv', converters={'HUC_6': lambda x: str(x)})
AgriCensus_dfHUC6 = AgriCensus_dfHUC6.set_index('HUC_6')
AgriCensus_dfHUC6 = AgriCensus_dfHUC6.fillna(0)
AgriCensus_dfHUC6

,Year,Corn,Soybeans,Small grains,Cotton,Rice,Vegetables,Orchards,Greenhouse,Other,Total,Sum
HUC_6,,,,,,,,,,,,
010100,2017.0,1.603983,1.097324,32.410438,0.000000,0.0,45.375313,0.119883,0.009626,19.383433,114278.0,100.0
010200,2017.0,19.464244,0.000000,2.460102,0.000000,0.0,7.260455,0.866292,0.121954,69.826952,47559.0,100.0
010300,2017.0,19.461339,0.000000,0.387402,0.000000,0.0,1.002221,0.470700,0.257827,78.420510,75632.0,100.0
010400,2017.0,13.170973,0.000000,0.000000,0.000000,0.0,3.518183,3.485272,0.217212,79.608359,30385.0,100.0
010500,2017.0,0.283779,0.000000,0.000000,0.000000,0.0,2.016052,0.757325,0.268111,96.674733,57439.0,100.0
010600,2017.0,7.383735,0.000000,0.344940,0.000000,0.0,7.736791,1.738901,4.425371,78.370262,49284.0,100.0
010700,2017.0,8.926910,0.000000,0.000000,0.000000,0.0,8.042245,5.441147,2.584682,75.005016,54823.0,100.0
010801,2017.0,15.850281,0.000000,0.000000,0.000000,0.0,1.384609,0.805292,0.170919,81.788898,121695.0,100.0
010802,2017.0,13.206884,0.458872,0.525143,0.000000,0.0,15.564503,2.840590,4.299542,63.104466,99592.0,100.0


In [24]:
AgriCensus_Point = AgriCensus_dfHUC6.loc[HUC6ContPoint]
AgriCensus_Point

Year            2.017000e+03
Corn            4.628742e+01
Soybeans        1.303660e+01
Small grains    2.681215e+01
Cotton          0.000000e+00
Rice            0.000000e+00
Vegetables      1.369653e-01
Orchards        1.337244e-03
Greenhouse      3.272315e-03
Other           1.372225e+01
Total           6.356356e+06
Sum             1.000000e+02
Name: 102500, dtype: float64

In [26]:
# Import databases
PlantNutrientUptake_df = pd.read_csv('DatabasesClean/PlantNutrientUptake.csv', index_col='Crop')
ForestsPlantNutrientUptake = pd.read_csv('DatabasesClean/ForestsPlantNutrientUptake.csv', index_col='Item')
SmallGrainsPlantNutrientUptake = pd.read_csv('DatabasesClean/SmallGrainsPlantNutrientUptake.csv', index_col='Item')
VegetablesPlantNutrientUptake = pd.read_csv('DatabasesClean/Vegetables_GreenhousePlantNutrientUptake.csv', index_col='Item')
OtherCropsPlantNutrientUptake = pd.read_csv('DatabasesClean/OtherCropsPlantNutrientUptake.csv', index_col='Item')
PasturePlantNutrientUptake = pd.read_csv('DatabasesClean/PasturePlantNutrientUptake.csv', index_col='Item')

Nutrients uptake for each land type is determined as follows:

$P_{uptake} = Area_{total} \cdot \sum_{j} \left(LandCoverFrac_{j} \cdot \sum_{i} \left( FracCrop_{i} \cdot \left(Yield_{i} \cdot P_{content \ i} \right) \right) \right), \ j 	\in \{\text{Land uses}\}, \ i \in \{\text{Crops}\} $

For croplands, the land is divided in the portion occupied by corn, soybeans, small grains, cotton, rice, vegetables, orchards, greenhouse and other crops (namely oil crops, sugar crops, and fruits). 

In [51]:
# Kg/Acre
Puptake_PFOR = 0.4535924*(ureg.kg/ureg.lb)*LandCover_Point['PFOR']/100 * ForestsPlantNutrientUptake.loc['P'].item()/100 * ForestsPlantNutrientUptake.loc['PoundsAcre'].item()*(ureg.lb/ureg.acre)
Puptake_PWETL = LandCover_Point['PWETL']/100 * Wetlands_Preg
Puptake_PAGC= 0.4535924*(ureg.kg/ureg.lb)*LandCover_Point['PAGC']/100 * (AgriCensus_Point['Corn']/100 * 
                                         (PlantNutrientUptake_df.loc['Corn','P'].item()/100 * PlantNutrientUptake_df.loc['Corn','PoundsAcre'].item()*(ureg.lb/ureg.acre)+
                                         PlantNutrientUptake_df.loc['CornStraw','P'].item()/100 * PlantNutrientUptake_df.loc['CornStraw','PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                        +
                                         AgriCensus_Point['Soybeans']/100 * 
                                         (PlantNutrientUptake_df.loc['Soybeans','P'].item()/100 * PlantNutrientUptake_df.loc['Soybeans','PoundsAcre'].item()*(ureg.lb/ureg.acre)+
                                         PlantNutrientUptake_df.loc['SoybeansStraw','P'].item()/100 * PlantNutrientUptake_df.loc['SoybeansStraw','PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                          +
                                         AgriCensus_Point['Small grains']/100 * 
                                         (SmallGrainsPlantNutrientUptake.loc['P'].item()/100 * SmallGrainsPlantNutrientUptake.loc['PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                         +
                                         AgriCensus_Point['Cotton']/100 * 
                                         (PlantNutrientUptake_df.loc['Cotton','P'].item()/100 * PlantNutrientUptake_df.loc['Cotton','PoundsAcre'].item()*(ureg.lb/ureg.acre)+
                                         PlantNutrientUptake_df.loc['CottonStraw','P'].item()/100 * PlantNutrientUptake_df.loc['CottonStraw','PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                         +
                                         AgriCensus_Point['Rice']/100 * 
                                         (PlantNutrientUptake_df.loc['Rice','P'].item()/100 * PlantNutrientUptake_df.loc['Rice','PoundsAcre'].item()*(ureg.lb/ureg.acre)+
                                         PlantNutrientUptake_df.loc['RiceStraw','P'].item()/100 * PlantNutrientUptake_df.loc['RiceStraw','PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                         +
                                         AgriCensus_Point['Vegetables']/100 * 
                                         (VegetablesPlantNutrientUptake.loc['P'].item() * VegetablesPlantNutrientUptake.loc['PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                         +
                                         AgriCensus_Point['Orchards']/100 * 
                                         (PlantNutrientUptake_df.loc['Apples','P'].item()/100 * PlantNutrientUptake_df.loc['Apples','PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                         +
                                         AgriCensus_Point['Greenhouse']/100 * 
                                         (VegetablesPlantNutrientUptake.loc['P'].item()/100 * VegetablesPlantNutrientUptake.loc['PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                         +
                                         AgriCensus_Point['Other']/100 * 
                                         (OtherCropsPlantNutrientUptake.loc['P'].item()/100 * OtherCropsPlantNutrientUptake.loc['PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                        )

Puptake_PAGP = 0.4535924*(ureg.kg/ureg.lb)*LandCover_Point['PAGP']/100 * PasturePlantNutrientUptake.loc['P'].item()/100 * PasturePlantNutrientUptake.loc['PoundsAcre'].item()*(ureg.lb/ureg.acre)
Puptake_PDEV = 0.4535924*(ureg.kg/ureg.lb)*LandCover_Point['PDEV']*0*(ureg.lb/ureg.acre)
Puptake_PDEV

0.0 <Unit('kilogram / acre')>

In [53]:
TotalHUC_PUptake = AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES'].item()*ureg.acre*(Puptake_PFOR+Puptake_PWETL+Puptake_PAGC+Puptake_PAGP+Puptake_PDEV)
TotalHUC_PUptake
# AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES']
# AreaHUC8.loc[HUC8ContPoint]

9900022.679505002 <Unit('kilogram')>

In [5]:
#Check with MRW watershed 05040004
LandCover_dfHUC8 = pd.read_csv('DatabasesClean/LandCover_HUC8_reconciliated.csv', converters={'HUC_8': lambda x: str(x), 'HUC_6': lambda x: str(x)})
LandCover_dfHUC8 = LandCover_dfHUC8.set_index('HUC_8')
LandCover_Point = LandCover_dfHUC8.loc['11120202']

AgriCensus_dfHUC6 = pd.read_csv('DatabasesClean/Agricensus.csv', converters={'HUC_6': lambda x: str(x)})
AgriCensus_dfHUC6 = AgriCensus_dfHUC6.set_index('HUC_6')
AgriCensus_dfHUC6 = AgriCensus_dfHUC6.fillna(0)
AgriCensus_Point = AgriCensus_dfHUC6.loc['111202']

poly  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
AreaHUC8 = poly.set_index('HUC_8', drop=False)[['AREA_ACRES','HUC_8']]

PlantNutrientUptake_df = pd.read_csv('DatabasesClean/PlantNutrientUptake.csv', index_col='Crop')
ForestsPlantNutrientUptake = pd.read_csv('DatabasesClean/ForestsPlantNutrientUptake.csv', index_col='Item')
SmallGrainsPlantNutrientUptake = pd.read_csv('DatabasesClean/SmallGrainsPlantNutrientUptake.csv', index_col='Item')
VegetablesPlantNutrientUptake = pd.read_csv('DatabasesClean/Vegetables_GreenhousePlantNutrientUptake.csv', index_col='Item')
OtherCropsPlantNutrientUptake = pd.read_csv('DatabasesClean/OtherCropsPlantNutrientUptake.csv', index_col='Item')
PasturePlantNutrientUptake = pd.read_csv('DatabasesClean/PasturePlantNutrientUptake.csv', index_col='Item')

Wetlands_Preg = 0.77*1E-3/0.0002471052 #Kg/Acre

# P uptake forest
Puptake_PFOR = 0.4535924*LandCover_Point['PFOR']/100 * ForestsPlantNutrientUptake.loc['P'].astype(float)/100 * ForestsPlantNutrientUptake.loc['PoundsAcre'].astype(float)
# P uptake wetlands
Puptake_PWETL = LandCover_Point['PWETL']/100 * Wetlands_Preg
# P uptake croplands
Puptake_PAGC= 0.4535924*LandCover_Point['PAGC']/100 * (AgriCensus_Point['Corn']/100 * 
                                         (PlantNutrientUptake_df.loc['Corn','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Corn','PoundsAcre'].astype(float)+
                                         PlantNutrientUptake_df.loc['CornStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['CornStraw','PoundsAcre'].astype(float))
                                        +
                                         AgriCensus_Point['Soybeans']/100 * 
                                         (PlantNutrientUptake_df.loc['Soybeans','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Soybeans','PoundsAcre'].astype(float)+
                                         PlantNutrientUptake_df.loc['SoybeansStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['SoybeansStraw','PoundsAcre'].astype(float))
                                          +
                                         AgriCensus_Point['Small grains']/100 * 
                                         (SmallGrainsPlantNutrientUptake.loc['P'].astype(float)/100 * SmallGrainsPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Cotton']/100 * 
                                         (PlantNutrientUptake_df.loc['Cotton','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Cotton','PoundsAcre'].astype(float)+
                                         PlantNutrientUptake_df.loc['CottonStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['CottonStraw','PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Rice']/100 * 
                                         (PlantNutrientUptake_df.loc['Rice','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Rice','PoundsAcre'].astype(float)+
                                         PlantNutrientUptake_df.loc['RiceStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['RiceStraw','PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Vegetables']/100 * 
                                         (VegetablesPlantNutrientUptake.loc['P'].astype(float) * VegetablesPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Orchards']/100 * 
                                         (PlantNutrientUptake_df.loc['Apples','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Apples','PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Greenhouse']/100 * 
                                         (VegetablesPlantNutrientUptake.loc['P'].astype(float)/100 * VegetablesPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Other']/100 * 
                                         (OtherCropsPlantNutrientUptake.loc['P'].astype(float)/100 * OtherCropsPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                        )
# P uptake pasture
Puptake_PAGP = 0.4535924*LandCover_Point['PAGP']/100 * PasturePlantNutrientUptake.loc['P'].astype(float)/100 * PasturePlantNutrientUptake.loc['PoundsAcre'].astype(float)
# P uptake development areas
Puptake_PDEV = 0.4535924*LandCover_Point['PDEV']/100*0

TotalHUC_PUptake = AreaHUC8.loc['11120202', 'AREA_ACRES'].astype(float)*float(Puptake_PFOR+Puptake_PWETL+Puptake_PAGC+Puptake_PAGP+Puptake_PDEV)
# TotalHUC_PUptake = AreaHUC8.loc['05040004', 'AREA_ACRES'].astype(float)*float(Puptake_PFOR+Puptake_PWETL+Puptake_PAGC+Puptake_PAGP+Puptake_PDEV)
TotalHUC_PUptake/364

5141.141278766131

In [6]:
type(TotalHUC_PUptake)


numpy.float64

# Module definition

In [57]:
# Joining everything in a function
# TotalHUC_PUptake_totalHUCs = []

def PUptake(HUC8ContPoint, HUC6ContPoint):
#     input_point = np.array([float(latitude), float(longitude)]) 

#     #input_point = np.array([39.014908, -98.010465])

#     input_point_df = pd.DataFrame(
#         {'Name': ['CAFO1'],
#          'Latitude': [input_point[0]],
#          'Longitude': [input_point[1]]})

#     poly  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
#     point = geopandas.GeoDataFrame(input_point_df, geometry=geopandas.points_from_xy(input_point_df.Longitude, input_point_df.Latitude))

#     poly.crs = {'init' :'epsg:4326'}
#     point.crs = {'init' :'epsg:4326'}

#     pointInPolys = sjoin(point, poly, how='left')

#     HUC8ContPoint = pointInPolys['HUC_8'].values[0]
    
#   LandCover_dfHUC8 = LandCover_dfHUC8.set_index('HUC_8')
    LandCover_Point = LandCover_dfHUC8.loc[HUC8ContPoint]
    AgriCensus_Point = AgriCensus_dfHUC6.loc[HUC6ContPoint]
    
    
    # P uptake forest
    Puptake_PFOR = AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES'].item()*ureg.acre * 0.4535924*(ureg.kg/ureg.lb)*LandCover_Point['PFOR'].item()/100 * ForestsPlantNutrientUptake.loc['P'].item()/100 * ForestsPlantNutrientUptake.loc['PoundsAcre'].item()*(ureg.lb/ureg.acre)
    # P uptake wetlands
    Puptake_PWETL = AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES'].item()*ureg.acre * LandCover_Point['PWETL'].item()/100 * Wetlands_Preg
    # P uptake croplands
    Puptake_PAGC= AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES'].item()*ureg.acre * 0.4535924*(ureg.kg/ureg.lb)*LandCover_Point['PAGC'].item()/100 * (AgriCensus_Point['Corn'].item()/100 * 
                                             (PlantNutrientUptake_df.loc['Corn','P'].item()/100 * PlantNutrientUptake_df.loc['Corn','PoundsAcre'].item()*(ureg.lb/ureg.acre)+
                                             PlantNutrientUptake_df.loc['CornStraw','P'].item()/100 * PlantNutrientUptake_df.loc['CornStraw','PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                             +
                                             AgriCensus_Point['Soybeans'].item()/100 * 
                                             (PlantNutrientUptake_df.loc['Soybeans','P'].item()/100 * PlantNutrientUptake_df.loc['Soybeans','PoundsAcre'].item()*(ureg.lb/ureg.acre)+
                                             PlantNutrientUptake_df.loc['SoybeansStraw','P'].item()/100 * PlantNutrientUptake_df.loc['SoybeansStraw','PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                             +
                                             AgriCensus_Point['Small grains'].item()/100 * 
                                             (SmallGrainsPlantNutrientUptake.loc['P'].item()/100 * SmallGrainsPlantNutrientUptake.loc['PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                             +
                                             AgriCensus_Point['Cotton'].item()/100 * 
                                             (PlantNutrientUptake_df.loc['Cotton','P'].item()/100 * PlantNutrientUptake_df.loc['Cotton','PoundsAcre'].item()*(ureg.lb/ureg.acre)+
                                             PlantNutrientUptake_df.loc['CottonStraw','P'].item()/100 * PlantNutrientUptake_df.loc['CottonStraw','PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                             +
                                             AgriCensus_Point['Rice'].item()/100 * 
                                             (PlantNutrientUptake_df.loc['Rice','P'].item()/100 * PlantNutrientUptake_df.loc['Rice','PoundsAcre'].item()*(ureg.lb/ureg.acre)+
                                             PlantNutrientUptake_df.loc['RiceStraw','P'].item()/100 * PlantNutrientUptake_df.loc['RiceStraw','PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                             +
                                             AgriCensus_Point['Vegetables'].item()/100 * 
                                             (VegetablesPlantNutrientUptake.loc['P'].item() * VegetablesPlantNutrientUptake.loc['PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                             +
                                             AgriCensus_Point['Orchards'].item()/100 * 
                                             (PlantNutrientUptake_df.loc['Apples','P'].item()/100 * PlantNutrientUptake_df.loc['Apples','PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                             +
                                             AgriCensus_Point['Greenhouse'].item()/100 * 
                                             (VegetablesPlantNutrientUptake.loc['P'].item()/100 * VegetablesPlantNutrientUptake.loc['PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                             +
                                             AgriCensus_Point['Other'].item()/100 * 
                                             (OtherCropsPlantNutrientUptake.loc['P'].item()/100 * OtherCropsPlantNutrientUptake.loc['PoundsAcre'].item()*(ureg.lb/ureg.acre))
                                             )

    # P uptake pasture
    Puptake_PAGP = AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES'].item()*ureg.acre * 0.4535924*(ureg.kg/ureg.lb)*LandCover_Point['PAGP'].item()/100 * PasturePlantNutrientUptake.loc['P'].item()/100 * PasturePlantNutrientUptake.loc['PoundsAcre'].item()*(ureg.lb/ureg.acre)
    # P uptake development areas
    Puptake_PDEV = AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES'].item()*ureg.acre * 0.4535924*(ureg.kg/ureg.lb)*LandCover_Point['PDEV'].item()/100 * 0 *(ureg.lb/ureg.acre)
    
    # Total P uptake kg/acre
    TotalHUC_PUptake = (Puptake_PFOR+Puptake_PWETL+Puptake_PAGC+Puptake_PAGP+Puptake_PDEV)
    
    # Anthropogenic P uptake
    AnthropogenicHUC_PUptake = (Puptake_PWETL+Puptake_PAGC+Puptake_PAGP+Puptake_PDEV)
    
#    TotalHUC_PUptake = TotalHUC_PUptake.to_numpy()
    TotalHUC_PUptake = TotalHUC_PUptake
    TotalHUC_PUptake_totalHUCs.append(TotalHUC_PUptake)
    
    AnthropogenicHUC_PUptake = AnthropogenicHUC_PUptake
    AnthropogenicHUC_PUptake_totalHUCs.append(AnthropogenicHUC_PUptake)
    
    Puptake_PFOR_HUC = Puptake_PFOR
    Puptake_PFOR_HUC_totalHUCs.append(Puptake_PFOR_HUC)
    
    Puptake_PWETL_HUC = Puptake_PWETL
    Puptake_PWETL_HUC_totalHUCs.append(Puptake_PWETL_HUC)
    
    Puptake_PAGC_HUC = Puptake_PAGC
    Puptake_PAGC_HUC_totalHUCs.append(Puptake_PAGC_HUC)
    
    Puptake_PAGP_HUC =  Puptake_PAGP
    Puptake_PAGP_HUC_totalHUCs.append(Puptake_PAGP_HUC)
    
    Puptake_PDEV_HUC = Puptake_PDEV
    Puptake_PDEV_HUC_totalHUCs.append(Puptake_PDEV_HUC)
                                      
    Area_HUC = AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES'].item()*ureg.acre
    Area_HUC_totalHUCs.append(Area_HUC)
#     return TotalHUC_PUptake

In [58]:
LandCover_dfHUC8 = pd.read_csv('DatabasesClean/LandCover_HUC8_reconciliated.csv', converters={'HUC_8': lambda x: str(x), 'HUC_6': lambda x: str(x)})
LandCover_dfHUC8 = LandCover_dfHUC8.set_index('HUC_8')

AgriCensus_dfHUC6 = pd.read_csv('DatabasesClean/Agricensus_reconciliated.csv', converters={'HUC_6': lambda x: str(x)})
AgriCensus_dfHUC6 = AgriCensus_dfHUC6.set_index('HUC_6')
AgriCensus_dfHUC6 = AgriCensus_dfHUC6.fillna(0)

poly  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
poly['HUC_8'] = poly['HUC_8'].astype(str)
AreaHUC8 = poly.set_index('HUC_8', drop=False)[['AREA_ACRES','HUC_8']]



# Import databases
PlantNutrientUptake_df = pd.read_csv('DatabasesClean/PlantNutrientUptake.csv', index_col='Crop')
ForestsPlantNutrientUptake = pd.read_csv('DatabasesClean/ForestsPlantNutrientUptake.csv', index_col='Item')
SmallGrainsPlantNutrientUptake = pd.read_csv('DatabasesClean/SmallGrainsPlantNutrientUptake.csv', index_col='Item')
VegetablesPlantNutrientUptake = pd.read_csv('DatabasesClean/Vegetables_GreenhousePlantNutrientUptake.csv', index_col='Item')
OtherCropsPlantNutrientUptake = pd.read_csv('DatabasesClean/OtherCropsPlantNutrientUptake.csv', index_col='Item')
PasturePlantNutrientUptake = pd.read_csv('DatabasesClean/PasturePlantNutrientUptake.csv', index_col='Item')

Wetlands_Preg = 0.77*1E-3/0.0002471052*(ureg.kg/ureg.acre) #Kg/Acre

TotalHUC_PUptake_totalHUCs = []
AnthropogenicHUC_PUptake_totalHUCs = []
Puptake_PFOR_HUC_totalHUCs = []
Puptake_PWETL_HUC_totalHUCs = []
Puptake_PAGC_HUC_totalHUCs = []
Puptake_PAGP_HUC_totalHUCs = []
Puptake_PDEV_HUC_totalHUCs = []
Area_HUC_totalHUCs = []
counter_HUC8 = []

HUC_8_array =  AreaHUC8['HUC_8'].to_numpy()
HUC_8_array_iter = iter(HUC_8_array)
for HUC8 in HUC_8_array:
    HUC8ContPoint = HUC8
    HUC6ContPoint = HUC8ContPoint[:6]
    PUptake(HUC8ContPoint, HUC6ContPoint)
    counter_HUC8 = HUC8
    
AnthropogenicHUC_PUptake_df = pd.DataFrame(AnthropogenicHUC_PUptake_totalHUCs, columns =['AnthropogenicHUC_PUptake_year'])
AnthropogenicHUC_PUptake_df['AnthropogenicHUC_PUptake_day'] = AnthropogenicHUC_PUptake_df['AnthropogenicHUC_PUptake_year']/365
AnthropogenicHUC_PUptake_df['HUC_8'] = HUC_8_array
AnthropogenicHUC_PUptake_df['AREA_ACRES'] = Area_HUC_totalHUCs
AnthropogenicHUC_PUptake_df['PFOR_year'] = Puptake_PFOR_HUC_totalHUCs
AnthropogenicHUC_PUptake_df['PWETL_year'] = Puptake_PWETL_HUC_totalHUCs
AnthropogenicHUC_PUptake_df['PAGC_year'] = Puptake_PAGC_HUC_totalHUCs
AnthropogenicHUC_PUptake_df['PAGP_year'] = Puptake_PAGP_HUC_totalHUCs
AnthropogenicHUC_PUptake_df['PDEV_year'] = Puptake_PDEV_HUC_totalHUCs
AnthropogenicHUC_PUptake_df['PFOR_day'] = AnthropogenicHUC_PUptake_df['PFOR_year']/365
AnthropogenicHUC_PUptake_df['PWETL_day'] = AnthropogenicHUC_PUptake_df['PWETL_year']/365
AnthropogenicHUC_PUptake_df['PAGC_day'] = AnthropogenicHUC_PUptake_df['PAGC_year']/365
AnthropogenicHUC_PUptake_df['PAGP_day'] = AnthropogenicHUC_PUptake_df['PAGP_year']/365
AnthropogenicHUC_PUptake_df['PDEV_day'] = AnthropogenicHUC_PUptake_df['PDEV_year']/365
AnthropogenicHUC_PUptake_df.to_csv('DatabasesClean/AnthropogenicHUC_PUptake.csv', index=False)

In [66]:
AnthropogenicHUC_PUptake_df

,AnthropogenicHUC_PUptake_year,AnthropogenicHUC_PUptake_day,HUC_8,AREA_ACRES,PFOR_year,PWETL_year,PAGC_year,PAGP_year,PDEV_year,PFOR_day,PWETL_day,PAGC_day,PAGP_day,PDEV_day
0,675635.9040592754 kilogram,1851.0572713952752 kilogram,04150306,551248.201671 acre,3644251.000058685 kilogram,268541.37221826275 kilogram,285622.0868151912 kilogram,121472.4450258214 kilogram,0.0 kilogram,9984.249315229274 kilogram,735.72978689935 kilogram,782.5262652470992 kilogram,332.80121924882576 kilogram,0.0 kilogram
1,2562009.2816784023 kilogram,7019.203511447678 kilogram,05010001,1648037.09494 acre,11259172.088442612 kilogram,145882.43879458195 kilogram,1620257.604296994 kilogram,795869.2385868265 kilogram,0.0 kilogram,30847.046817650993 kilogram,399.67791450570394 kilogram,4439.061929580806 kilogram,2180.4636673611685 kilogram,0.0 kilogram
2,1023203.1863701721 kilogram,2803.29640101417 kilogram,05050002,1078369.23741 acre,7153849.015558119 kilogram,1642.63126850532 kilogram,209682.71532414388 kilogram,811877.8397775228 kilogram,0.0 kilogram,19599.586343994848 kilogram,4.500359639740602 kilogram,574.4731926688874 kilogram,2224.322848705542 kilogram,0.0 kilogram
3,19131048.38439795 kilogram,52413.83119013137 kilogram,05080001,1598490.90472 acre,1255903.209072969 kilogram,13943.547102700877 kilogram,18632176.774646554 kilogram,484928.06264869636 kilogram,0.0 kilogram,3440.830709788956 kilogram,38.201498911509255 kilogram,51047.0596565659 kilogram,1328.5700346539627 kilogram,0.0 kilogram
4,2602242.0410392415 kilogram,7129.430249422579 kilogram,05090203,900352.682902 acre,3810896.9281068454 kilogram,7052.223960819485 kilogram,1852755.263408504 kilogram,742434.5536699182 kilogram,0.0 kilogram,10440.81350166259 kilogram,19.32116153649174 kilogram,5076.041817557545 kilogram,2034.0672703285431 kilogram,0.0 kilogram
5,10934182.285603134 kilogram,29956.66379617297 kilogram,05120111,1291942.51424 acre,2755682.761896368 kilogram,47534.06644424675 kilogram,10642297.686782395 kilogram,244350.5323764926 kilogram,0.0 kilogram,7549.815786017447 kilogram,130.23031902533356 kilogram,29156.979963787384 kilogram,669.4535133602537 kilogram,0.0 kilogram
6,5805836.650601424 kilogram,15906.401782469655 kilogram,05120115,671319.553201 acre,1183173.0880551462 kilogram,6590.778608938963 kilogram,5363306.264774814 kilogram,435939.60721767147 kilogram,0.0 kilogram,3241.5701042606743 kilogram,18.056927695723186 kilogram,14693.98976650634 kilogram,1194.3550882675931 kilogram,0.0 kilogram
7,3525359.6074654693 kilogram,9658.519472508135 kilogram,05140102,946114.578675 acre,3534004.941664719 kilogram,55323.65622233027 kilogram,2087346.2847341993 kilogram,1382689.6665089398 kilogram,0.0 kilogram,9682.205319629367 kilogram,151.57166088309663 kilogram,5718.756944477258 kilogram,3788.1908671477804 kilogram,0.0 kilogram
8,3671779.77959272 kilogram,10059.67062902115 kilogram,05140206,600162.803804 acre,1550654.3398230628 kilogram,87243.95069790893 kilogram,3147503.0254935767 kilogram,437032.80340123386 kilogram,0.0 kilogram,4248.368054309761 kilogram,239.02452246002449 kilogram,8623.295960256375 kilogram,1197.3501463047503 kilogram,0.0 kilogram
9,2019384.682690212 kilogram,5532.5607744937315 kilogram,06020002,1324768.79325 acre,8527513.401135378 kilogram,13239.951600441127 kilogram,1245447.1581924716 kilogram,760697.5728972993 kilogram,0.0 kilogram,23363.050414069527 kilogram,36.27384000120857 kilogram,3412.1839950478675 kilogram,2084.1029394446555 kilogram,0.0 kilogram


In [68]:
# from folium.plugins import MousePosition, Search
# import branca

# HUC8_json = poly.to_crs(epsg='4326').to_json()

# AnthropogenicHUC_PUptake_df = pd.read_csv('DatabasesClean/AnthropogenicHUC_PUptake.csv', converters={'HUC_8': lambda x: str(x)})
# AnthropogenicHUC_PUptake_df_GEOjson = poly.merge(AnthropogenicHUC_PUptake_df, on='HUC_8')

# m_Anthropogenic_PUptake = folium.Map(location=[48, -102], zoom_start=6)

# bins = list(AnthropogenicHUC_PUptake_df_GEOjson['AnthropogenicHUC_PUptake_year'].quantile([0, 0.1, 0.2, 0.3,0.5,0.6,0.7,0.8,0.9, 1]))
# # colormap = branca.colormap.linear.YlGnBu_09.scale(0, 30)

# # colormap = branca.colormap.linear.YlOrRd_09.scale(0, 8500)
# # colormap = colormap.to_step(index=[0, 1000, 3000, 5000, 8500])
# # colormap.caption = 'Incidents of Crime in Victoria (year ending June 2018)'
# # colormap.add_to(world_map)

# colormap = branca.colormap.linear.YlGnBu_09.scale(
#     AnthropogenicHUC_PUptake_df_GEOjson['AnthropogenicHUC_PUptake_year'].min(),
#     AnthropogenicHUC_PUptake_df_GEOjson['AnthropogenicHUC_PUptake_year'].max())

# # Anthropogenic_P_Uptake_layer = folium.Choropleth(
# #     geo_data=HUC8_json,
# #     name='Anthropogenic P Uptake',
# #     data=AnthropogenicHUC_PUptake_df_GEOjson,
# #     columns=['HUC_8', 'AnthropogenicHUC_PUptake_year'],
# #     key_on='feature.properties.HUC_8',
# # #     bins = bins,
# #     fill_color=colorscale,
# #     fill_opacity=0.7,
# #     line_opacity=0.2,
# #     nan_fill_color='grey',
# #     nan_fill_opacity=0.7,
# #     legend_name=r"Anthropogenic P uptake (kg/year)",
# #     highlight=True,
# # ).add_to(m_Anthropogenic_PUptake)

# # folium.GeoJson(
# #     AnthropogenicHUC_PUptake_df_GEOjson.to_json(),
# #     style_function=lambda feature: {
# #         'fillOpacity': 0,
# #         'color': 'black',
# #         'weight': 0.1,
# #     },
# #     tooltip=folium.features.GeoJsonTooltip(
# #         fields=['HUC_8', 'AnthropogenicHUC_PUptake_year'],
# #         aliases=['HUC8 watershed', 'Anthropogenic P uptake (kg/year)'], 
# #         localize=True)
# #     ).add_to(m_Anthropogenic_PUptake)



# AnthropogenicHUC_PUptake_year_dict = AnthropogenicHUC_PUptake_df_GEOjson.set_index('HUC_8')['AnthropogenicHUC_PUptake_year']

# folium.GeoJson(
#     AnthropogenicHUC_PUptake_df_GEOjson.to_json(),
#     name='Anthropogenic P Uptake',
#     style_function=lambda feature: {
#         'fillColor': colormap(AnthropogenicHUC_PUptake_year_dict[feature['properties']['HUC_8']]),
#         'fillOpacity': 0.9,
#         'color': 'black',
#         'weight': 0.1,
#     },
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['HUC_8', 'AnthropogenicHUC_PUptake_year'],
#         aliases=['HUC8 watershed', 'Anthropogenic P uptake (kg/year)'], 
#         localize=True)
#     ).add_to(m_Anthropogenic_PUptake)

# colormap.caption = 'Anthropogenic P uptake (kg/year)'
# colormap.add_to(m_Anthropogenic_PUptake)

# folium.LayerControl().add_to(m_Anthropogenic_PUptake)

# formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"

# MousePosition(
#     position='bottomright',
#     separator=' | ',
#     empty_string='NaN',
#     lng_first=True,
#     num_digits=20,
#     prefix='Coordinates:',
#     lat_formatter=formatter,
#     lng_formatter=formatter,
# ).add_to(m_Anthropogenic_PUptake)

# HUC8search = Search(
#     layer=Anthropogenic_P_Uptake_layer,
#     geom_type='Polygon',
#     placeholder='Search for a HUC8 watershed',
#     collapsed=False,
#     search_label='HUC_8',
#     weight=3
# ).add_to(m_Anthropogenic_PUptake)

# m_Anthropogenic_PUptake.save('m_Anthropogenic_PUptake_units.html')

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [46]:
# AnthropogenicHUC_PUptake_df = pd.read_csv('DatabasesClean/AnthropogenicHUC_PUptake.csv', converters={'HUC_8': lambda x: str(x)})
# AnthropogenicHUC_PUptake_df_GEOjson = poly.merge(AnthropogenicHUC_PUptake_df, on='HUC_8')
# # AnthropogenicHUC_PUptake_df_GEOjson = AnthropogenicHUC_PUptake_df 
# # AnthropogenicHUC_PUptake_df_GEOjson['geometry'] = poly['geometry']
# # AnthropogenicHUC_PUptake_df_GEOjson
# AnthropogenicHUC_PUptake_df_GEOjson

,HUC_8,HU_8_NAME,AREA_ACRES_x,AREA_SQKM,STATES,STORET_COU,STORET_RES,NWIS_COUNT,WSA_CPL_TO,WSA_CPL__1,WSA_CPL__2,WSA_CPL__3,WSA_NAP_TO,WSA_NAP__1,WSA_NAP__2,WSA_NAP__3,WSA_NPL_TO,WSA_NPL__1,WSA_NPL__2,WSA_NPL__3,WSA_SAP_TO,WSA_SAP__1,WSA_SAP__2,WSA_SAP__3,WSA_SPL_TO,WSA_SPL__1,WSA_SPL__2,WSA_SPL__3,WSA_TPL_TO,WSA_TPL__1,WSA_TPL__2,WSA_TPL__3,WSA_UMW_TO,WSA_UMW__1,WSA_UMW__2,WSA_UMW__3,WSA_WMT_TO,WSA_WMT__1,WSA_WMT__2,WSA_WMT__3,WSA_XER_TO,WSA_XER__1,WSA_XER__2,WSA_XER__3,NLA_CGP_TO,NLA_CGP__1,NLA_CGP__2,NLA_CGP__3,NLA_CPE_TO,NLA_CPE__1,NLA_CPE__2,NLA_CPE__3,NLA_GPM_TO,NLA_GPM__1,NLA_GPM__2,NLA_GPM__3,NLA_GPN_TO,NLA_GPN__1,NLA_GPN__2,NLA_GPN__3,NLA_NGL_TO,NLA_NGL__1,NLA_NGL__2,NLA_NGL__3,NLA_SAM_TO,NLA_SAM__1,NLA_SAM__2,NLA_SAM__3,NLA_SEP_TO,NLA_SEP__1,NLA_SEP__2,NLA_SEP__3,NLA_SGL_TO,NLA_SGL__1,NLA_SGL__2,NLA_SGL__3,NLA_TMP_TO,NLA_TMP__1,NLA_TMP__2,NLA_TMP__3,NLA_WMT_TO,NLA_WMT__1,NLA_WMT__2,NLA_WMT__3,NLA_XER_TO,NLA_XER__1,NLA_XER__2,NLA_XER__3,NLCD11A,NLCD11P,NLCD12A,NLCD12P,NLCD21A,NLCD21P,NLCD22A,NLCD22P,NLCD23A,NLCD23P,NLCD24A,NLCD24P,NLCD31A,NLCD31P,NLCD41A,NLCD41P,NLCD42A,NLCD42P,NLCD43A,NLCD43P,NLCD52A,NLCD52P,NLCD71A,NLCD71P,NLCD81A,NLCD81P,NLCD82A,NLCD82P,NLCD90A,NLCD90P,NLCD95A,NLCD95P,LIKELY_TO_,HAS_N_LIMI,HAS_P_LIMI,HAS_N_MONI,HAS_P_MONI,HAS_N_DMR_,HAS_P_DMR_,ATTAINS_30,ATTAINS_TM,Shape_Leng,Shape_Area,geometry,AnthropogenicHUC_PUptake_year,AnthropogenicHUC_PUptake_day,AREA_ACRES_y,PFOR_year,PWETL_year,PAGC_year,PAGP_year,PDEV_year,PFOR_day,PWETL_day,PAGC_day,PAGP_day,PDEV_day
0,04150306,St. Regis,5.512482e+05,2230.822092,NY,5.0,156.0,4.0,0.0,0.0,0.0,0.0,329.0,441.0,8.2,15.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,674.0,1174.0,16.5,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,828.0,1410.0,24.0,102.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67.4973,0.030497,0.0000,0.000000,25.6329,0.011582,2.6937,0.001217,0.3033,0.000137,0.0558,0.000025,1.3869,0.000627,1121.7753,0.506844,362.9259,0.163978,50.8455,0.022973,64.2096,0.029011,20.5236,0.009273,83.8224,0.037873,33.8562,0.015297,352.4211,0.159232,25.3251,0.011442,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,10.0,2.739500,0.253010,POLYGON ((-74.43569948699991 44.34982683300007...,6.756359e+05,1851.057271,5.512482e+05,3.644251e+06,2.685414e+05,2.856221e+05,1.214724e+05,0.0,9984.249315,735.729787,782.526265,332.801219,0.0
1,05010001,Upper Allegheny,1.648037e+06,6669.368806,"NY,PA",20.0,1681.0,25.0,0.0,0.0,0.0,0.0,329.0,441.0,8.2,15.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,674.0,1174.0,16.5,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,828.0,1410.0,24.0,102.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.3739,0.008902,0.0000,0.000000,170.2944,0.025534,38.0439,0.005704,14.2290,0.002133,2.5335,0.000380,16.0839,0.002412,4270.6368,0.640331,302.5584,0.045365,485.9163,0.072857,331.2000,0.049659,51.3369,0.007697,566.8920,0.084999,217.1736,0.032563,129.0528,0.019350,14.1255,0.002118,17.0,12.0,2.0,7.0,13.0,15.0,15.0,12.0,6.0,6.173681,0.724405,POLYGON ((-79.06824495299992 41.77853391200006...,2.562009e+06,7019.203511,1.648037e+06,1.125917e+07,1.458824e+05,1.620258e+06,7.958692e+05,0.0,30847.046818,399.677915,4439.061930,2180.463667,0.0
2,05050002,Middle New,1.078369e+06,4364.005019,"VA,WV",70.0,355.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,296.0,535.0,17.8,24.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.1091,0.005753,0.0000,0.000000,193.1715,0.044261,79.0614,0.018115,34.8129,0.007977,5.5368,0.001269,5.3442,0.001225,2902.8690,0.665131,213.4917,0.048917,82.2726,0.018851,3.9078,0.000895,116.5257,0.026699,685.1295,

# Antrophogenic P emissions

Agricultural emissions are main source of antrophogenic P emissions, due to commercial fertilizers application and the use of livestock organic waste nutrients from in croplands or uncontroled runoff to waterbodies. We have considered the other possible human-based sources of P emissions negigible compared with the agricultural releases (Lee and Bakshi, 2019. Energy-Water-CO2 Nexus of Fossil Fuel Based Power Generation, book carbon management, David A. Dzombak 2011. Nutrient Control in Large-Scale
U.S. Watersheds. The Chesapeake Bay and Northern Gulf of Mexico. The BRIDGE, NATIONAL ACADEMY OF ENGINEERING).

## Commercial fertilizers
### Nutrients content
Phosphorus and nitrogen content of several commercial phosphate fertilizers (Ohio State University (OSU) Extension, Ohio Agronomy Guide, 15th Edition, 2017.)

|      Fertilizer Material     | % P (as P2O5)* |  % N  |
|:----------------------------:|:--------------:|:-----:|
|     Single superphosphate    |      16-20     |   -   |
|     Triple superphosphate    |      44-48     |   -   |
| Monoammonium phosphate (MAP) |      48-61     | 10-12 |
|  Diammonium phosphate (DAP)  |       46       |   18  |
|    Ammonium polyphosphate    |       34       |   10  |

*: Phosphate is typically measured as P2O5.


The following values based on Ohio State University, 2019 are considered (Ohio State University (OSU) Extension, Ohio Agronomy Guide, 15th Edition, 2017.):

|      Fertilizer Material     | % P (as P2O5)* | % N |
|:----------------------------:|:--------------:|:---:|
|     Single superphosphate    |       18       |  -  |
|     Triple superphosphate    |       46       |  -  |
| Monoammonium phosphate (MAP) |      54.5      |  11 |
|  Diammonium phosphate (DAP)  |       46       |  18 |
|    Ammonium polyphosphate    |       34       |  10 |

*: Phosphate is typically measured as P2O5.

### Quantities
The amount of nutirents from manure for the United States were drawn from the IPNI NuGIS project database, which includes estimates based on fertilizers sales for 2014 provided by the Association of American Plant Food Control Officials (AAPFCO), fiting the data to the HUC8 watershed boundaries.

In [71]:
Fertilizer_df = pd.read_csv('PInputs/PInputs.csv', converters={'HUC_CODE': lambda x: str(x)})
for index, row in Fertilizer_df.iterrows():
    if len(row['HUC_CODE'])<8:
        row['HUC_CODE'] = '0' + row['HUC_CODE']
        Fertilizer_df.loc[index, 'HUC_CODE'] = row['HUC_CODE']
        
NutFertilizer = Fertilizer_df[['HUC_CODE', 'Tons_N_Farm_Fert', 'Tons_P2O5_Farm_Fert', 'Tons_K2O_Farm_Fert']]

# P2O5 to P
NutFertilizer['P_Fert'] = NutFertilizer['Tons_P2O5_Farm_Fert']/2.29

# K2O to K
NutFertilizer['K_Fert'] = NutFertilizer['Tons_K2O_Farm_Fert']/1.2

NutFertilizer['N_Fert'] = NutFertilizer['Tons_N_Farm_Fert']

#kg (from pound tons)
NutFertilizer[['N_Fert', 'P_Fert', 'K_Fert']] = NutFertilizer [['N_Fert', 'P_Fert', 'K_Fert']].apply(lambda x: x*2000*0.4535924)
        
# NutFertilizer.to_csv('DatabasesClean/NutFertilizer.csv', index=False)
NutFertilizer

/home/emh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/emh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/emh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

,HUC_CODE,Tons_N_Farm_Fert,Tons_P2O5_Farm_Fert,Tons_K2O_Farm_Fert,P_Fert,K_Fert,N_Fert
0,01010001,571.016454,389.809255,595.895728,1.544232e+05,4.504896e+05,518017
1,01010002,89.161934,60.834240,93.633475,2.409952e+04,7.078572e+04,80886.4
2,01010003,679.179020,463.770211,707.130823,1.837228e+05,5.345819e+05,616141
3,01010004,2411.329132,1646.452618,2512.172053,6.522431e+05,1.899170e+06,2.18752e+06
4,01010005,1206.046156,823.535862,1255.681324,3.262442e+05,9.492792e+05,1.09411e+06
5,01020001,9.780127,6.202406,17.685601,2.457087e+03,1.337009e+04,8872.38
6,01020002,4.957682,3.140983,9.164311,1.244302e+03,6.928103e+03,4497.53
7,01020003,247.572692,168.387131,268.782142,6.670666e+04,2.031959e+05,224594
8,01020004,45.077627,29.061350,75.009163,1.151267e+04,5.670598e+04,40893.7
9,01020005,106.416681,65.563180,227.826330,2.597289e+04,1.722338e+05,96539.6


In [25]:
NutFertilizer.set_index('HUC_CODE').loc['03090203']


Tons_N_Farm_Fert          1.118705
Tons_P2O5_Farm_Fert       0.214263
Tons_K2O_Farm_Fert        0.832603
P_Fert                   84.880501
K_Fert                  629.437106
N_Fert                 1014.872269
Name: 03090203, dtype: float64

## Manure
### Nutrients content
Phosphorus and nitrogen content of several manure types (Ohio State University (OSU) Extension, Ohio Agronomy Guide, 14th Edition, 2005.)

|  Manure Type | % P (as P2O5)* | % N* |
|:------------:|:--------------:|------|
| Dairy cattle |       1.8      | 4.3  |
|    Swine**   |       4.3      | 14.0 |
|  Poultry***  |        7       | 3.9  |

*: Percentages on a dry weight basis; **: Finishing; ***: Layers

### Quantities
The amount of nutirents from manure for the United States were drawn from the IPNI NuGIS project database, which includes estimates based on livestock numbers reported in census data, and reported excretion rates for different livestock. The data reported are estimated for 2014 adpating the very detailed methodology proposed by Kellog et al. (2000) (Manure Nutrients Relative to the Capacity of Cropland and Pastureland to Assimilate Nutrients, USDA, 2000) to the HUC8 watershed boundaries.

In [34]:
ManureGenerated_df = pd.read_csv('PInputs/PInputs.csv', converters={'HUC_CODE': lambda x: str(x)})
for index, row in ManureGenerated_df.iterrows():
    if len(row['HUC_CODE'])<8:
        row['HUC_CODE'] = '0' + row['HUC_CODE']
        ManureGenerated_df.loc[index, 'HUC_CODE'] = row['HUC_CODE']
        
NutManureGenerated = ManureGenerated_df[['HUC_CODE', 'Tons_N_Exc', 'Tons_P2O5_Exc', 'Tons_K2O_Exc']]

# P2O5 to P
NutManureGenerated['P_Exc'] = NutManureGenerated['Tons_P2O5_Exc']/2.29

# K2O to K
NutManureGenerated['K_Exc'] = NutManureGenerated['Tons_K2O_Exc']/1.2

NutManureGenerated['N_Exc'] = NutManureGenerated['Tons_N_Exc']

#kg (from pound tons)
NutManureGenerated[['N_Exc', 'P_Exc', 'K_Exc']] = NutManureGenerated [['N_Exc', 'P_Exc', 'K_Exc']].apply(lambda x: x*2000*0.4535924)
        
NutManureGenerated.to_csv('DatabasesClean/NutManureGenerated.csv', index=False)
NutManureGenerated

/home/emh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/emh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/emh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

,HUC_CODE,Tons_N_Exc,Tons_P2O5_Exc,Tons_K2O_Exc,P_Exc,K_Exc,N_Exc
0,01010001,48.801387,28.370166,34.065434,1.123886e+04,2.575304e+04,4.427188e+04
1,01010002,9.421921,4.839030,6.576900,1.916985e+03,4.972053e+03,8.547424e+03
2,01010003,49.392404,30.229807,34.477989,1.197556e+04,2.606492e+04,4.480804e+04
3,01010004,182.389360,109.508245,127.315495,4.338175e+04,9.624890e+04,1.654609e+05
4,01010005,87.708127,53.680313,61.223986,2.126549e+04,4.628456e+04,7.956748e+04
5,01020001,34.428979,11.519834,24.032885,4.563589e+03,1.816856e+04,3.123345e+04
6,01020002,21.207793,6.932089,14.803937,2.746151e+03,1.119159e+04,1.923939e+04
7,01020003,71.187351,28.650443,49.691785,1.134989e+04,3.756636e+04,6.458008e+04
8,01020004,108.836626,33.609245,75.972573,1.331432e+04,5.743430e+04,9.873493e+04
9,01020005,582.241727,258.486491,406.429374,1.023996e+05,3.072555e+05,5.282008e+05


In [3]:
# df["ID"] = '>' + df["ID"]
# # a = ManureGenerated_df['HUC_CODE'].where(len(ManureGenerated_df['HUC_CODE'])<8,ManureGenerated_df['HUC_CODE'])
# for index, row in ManureGenerated_df.iterrows():
#     if len(row['HUC_CODE'])<8:
#         row['HUC_CODE'] = '0' + row['HUC_CODE']
#         ManureGenerated_df.loc[index, 'HUC_CODE'] = row['HUC_CODE']
#         print(row['HUC_CODE'])

# NutManureGenerated_i = ManureGenerated_df[['HUC_CODE', 'Tons_N_Exc', 'Tons_P2O5_Exc', 'Tons_K2O_Exc']]

# # P2O5 to P
# NutManureGenerated_i['Tons_P_Exc'] = NutManureGenerated_i['Tons_P2O5_Exc']/2.29

# # K2O to K
# NutManureGenerated_i['Tons_K_Exc'] = NutManureGenerated_i['Tons_K2O_Exc']/1.2

# #kg
# NutManureGenerated = NutManureGenerated_i [['Tons_N_Exc', 'Tons_P2O5_Exc', 'Tons_K2O_Exc','Tons_P_Exc','Tons_K_Exc']].apply(lambda x: x*0.4535924)


        
# # AnthropogenicHUC_PUptake_df.to_csv('DatabasesClean/AnthropogenicHUC_PUptake.csv', index=False)
# NutManureGenerated

# # z = ('0' + ManureGenerated_df['HUC_CODE']).where(len(ManureGenerated_df['HUC_CODE'])<8 ,  ManureGenerated_df['HUC_CODE'])

/home/emh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/emh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Tons_N_Exc,Tons_P2O5_Exc,Tons_K2O_Exc,Tons_P_Exc,Tons_K_Exc
0,22.135938,12.868492,15.451822,5.619429,12.876518
1,4.273712,2.194947,2.983232,0.958492,2.486026
2,22.404019,13.712011,15.638954,5.987778,13.032462
3,82.730428,49.672108,57.749341,21.690877,48.124451
4,39.783740,24.348982,27.770735,10.632743,23.142279
5,15.616723,5.225309,10.901134,2.281794,9.084278
6,9.619693,3.144343,6.714953,1.373075,5.595794
7,32.290041,12.995623,22.539816,5.674945,18.783180
8,49.367467,15.244898,34.460582,6.657161,28.717151
9,264.100422,117.247508,184.353275,51.199785,153.627729


## Total antrophogenic P emissions emissions

In [10]:
NutFertilizer_df = pd.read_csv('DatabasesClean/NutFertilizer.csv', converters={'HUC_CODE': lambda x: str(x)})
NutManureGenerated_df = pd.read_csv('DatabasesClean/NutManureGenerated.csv', converters={'HUC_CODE': lambda x: str(x)})

# TotalEmissions['P'] = NutManureGenerated_df['P_Exc'] + NutFertilizer_df['P_Fert']
TotalEmissions = pd.DataFrame({'HUC_8':NutManureGenerated_df['HUC_CODE'],
                               'P':(NutManureGenerated_df['P_Exc'] + NutFertilizer_df['P_Fert']),
                               'N':(NutManureGenerated_df['N_Exc'] + NutFertilizer_df['N_Fert']),
                               'K':(NutManureGenerated_df['K_Exc'] + NutFertilizer_df['K_Fert']),})

TotalEmissions.to_csv('DatabasesClean/TotalEmissions.csv', index=False)

# Antrophogenic P balance and TES

In [16]:
TotalEmissions_df = pd.read_csv('DatabasesClean/TotalEmissions.csv', converters={'HUC_8': lambda x: str(x)})
AnthropogenicHUC_PUptake_df = pd.read_csv('DatabasesClean/AnthropogenicHUC_PUptake.csv', converters={'HUC_8': lambda x: str(x)})

AntrophogenicPBalance = pd.DataFrame({'HUC_8':TotalEmissions_df['HUC_8'],
                                       'P_emitted':TotalEmissions['P'],
                                       'P_uptake':AnthropogenicHUC_PUptake_df['AnthropogenicHUC_PUptake_year'],
                                       'P_balance':(TotalEmissions['P'] - AnthropogenicHUC_PUptake_df['AnthropogenicHUC_PUptake_year']),
                                       'P_TES':((AnthropogenicHUC_PUptake_df['AnthropogenicHUC_PUptake_year']-TotalEmissions['P'])/TotalEmissions['P'])})

AntrophogenicPBalance.to_csv('DatabasesClean/AntrophogenicPBalance.csv', index=False)

In [30]:
from folium.plugins import MousePosition, Search
import branca
import branca.colormap as cm

poly  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
poly['HUC_8'] = poly['HUC_8'].astype(str)
HUC8_json = poly.to_crs(epsg='4326').to_json()

AntrophogenicPBalance_df = pd.read_csv('DatabasesClean/AntrophogenicPBalance.csv', converters={'HUC_8': lambda x: str(x)})
AntrophogenicPBalance_df_GEOjson = poly.merge(AntrophogenicPBalance_df, on='HUC_8')

m_AntrophogenicPBalance = folium.Map(location=[48, -102], zoom_start=6)

# bins = list(AntrophogenicPBalance_df_GEOjson['P_TES'].quantile([0, 0.1, 0.2, 0.3,0.5,0.6,0.7,0.8,0.9, 1]))
# colormap = branca.colormap.linear.YlGnBu_09.scale(0, 30)

# colormap = branca.colormap.linear.YlOrRd_09.scale(0, 8500)
# colormap = colormap.to_step(index=[0, 1000, 3000, 5000, 8500])
# colormap.caption = 'Incidents of Crime in Victoria (year ending June 2018)'
# colormap.add_to(world_map)

# colormap = branca.colormap.linear.YlGnBu_09.scale(
#     AntrophogenicPBalance_df_GEOjson['P_TES'].min(),
#     AntrophogenicPBalance_df_GEOjson['P_TES'].max())

# colormap = cm.LinearColormap(colors=['red','blue'], index=[AntrophogenicPBalance_df_GEOjson['P_TES'].min(),0],
#                              vmin=AntrophogenicPBalance_df_GEOjson['P_TES'].min(),vmax=AntrophogenicPBalance_df_GEOjson['P_TES'].max())

colormap = cm.StepColormap(colors=['red','#32c7b4'], index=[AntrophogenicPBalance_df_GEOjson['P_TES'].min(),0],
                             vmin=AntrophogenicPBalance_df_GEOjson['P_TES'].min(),vmax=AntrophogenicPBalance_df_GEOjson['P_TES'].max())

# colormap.to_step(index=[-200, 0, 1200])


# Anthropogenic_P_Uptake_layer = folium.Choropleth(
#     geo_data=HUC8_json,
#     name='Anthropogenic P Uptake',
#     data=AnthropogenicHUC_PUptake_df_GEOjson,
#     columns=['HUC_8', 'AnthropogenicHUC_PUptake_year'],
#     key_on='feature.properties.HUC_8',
# #     bins = bins,
#     fill_color=colorscale,
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     nan_fill_color='grey',
#     nan_fill_opacity=0.7,
#     legend_name=r"Anthropogenic P uptake (kg/year)",
#     highlight=True,
# ).add_to(m_Anthropogenic_PUptake)

# folium.GeoJson(
#     AnthropogenicHUC_PUptake_df_GEOjson.to_json(),
#     style_function=lambda feature: {
#         'fillOpacity': 0,
#         'color': 'black',
#         'weight': 0.1,
#     },
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['HUC_8', 'AnthropogenicHUC_PUptake_year'],
#         aliases=['HUC8 watershed', 'Anthropogenic P uptake (kg/year)'], 
#         localize=True)
#     ).add_to(m_Anthropogenic_PUptake)



P_TES_dict = AntrophogenicPBalance_df_GEOjson.set_index('HUC_8')['P_TES']

AntrophogenicPBalance_layer = folium.GeoJson(
    AntrophogenicPBalance_df_GEOjson.to_json(),
    name='P TES',
    style_function=lambda feature: {
        'fillColor': colormap(P_TES_dict[feature['properties']['HUC_8']]),
        'fillOpacity': 0.9,
        'color': 'black',
        'weight': 0.1,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['HUC_8', 'P_TES'],
        aliases=['HUC8 watershed', 'P TES'], 
        localize=True)
    ).add_to(m_AntrophogenicPBalance)

colormap.caption = 'P TES'
colormap.add_to(m_AntrophogenicPBalance)

folium.LayerControl().add_to(m_AntrophogenicPBalance)

formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"

MousePosition(
    position='bottomright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(m_AntrophogenicPBalance)

HUC8search = Search(
    layer=AntrophogenicPBalance_layer,
    geom_type='Polygon',
    placeholder='Search for a HUC8 watershed',
    collapsed=False,
    search_label='HUC_8',
    weight=3
).add_to(m_AntrophogenicPBalance)

m_AntrophogenicPBalance.save('m_AntrophogenicPBalance.html')

In [19]:
5134884.62321*0.0155824536147267/100

800.141014581432